In [59]:
%autosave 60

import plotly.io as pio
### optional (fast_conv and fast_ssm) ###

## install cuda
# if you are on a devbox, use
# sudo apt install wget
# wget https://developer.download.nvidia.com/compute/cuda/12.0.0/local_installers/cuda_12.0.0_525.60.13_linux.run
# sudo sh cuda_12.0.0_525.60.13_linux.run
# add /usr/local/cuda-12.0/bin/ to PATH
# that gives u nvcc

## fast_conv=True
# if you want fast_conv=True you need to install
# https://github.com/Dao-AILab/causal-conv1d
# via
# pip install causal-conv1d

## fast_ssm=True
# if you want fast_ssm=True you need to install
# https://github.com/state-spaces/mamba
# via
# pip install mamba-ssm


### install plotly ###

# if plotly isn't working, i did
# install nvm from
# https://github.com/nvm-sh/nvm
# and activate it
# then i did
# nvm install node
# jupyter labextension install plotlywidget

# if the jupyter widgets aren't working i have no recommendations
# only god can help you

# Import stuff
import torch
from functools import partial
import transformer_lens.utils as utils
import plotly.express as px
torch.set_grad_enabled(False)
device = utils.get_device()
# from neel nanda's examples
def imshow(tensor, renderer=None, xaxis="", yaxis="", font_size=None, show=True, color_continuous_midpoint=0.0, **kwargs):
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=color_continuous_midpoint, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs)
    if not font_size is None:
        fig.update_layout(
          xaxis = dict(
            tickmode='array',
            tickvals = kwargs['x'],
            ticktext = kwargs['x'], 
            ),
           font=dict(size=font_size, color="black"))
    if show:
        fig.show(renderer)
    else:
        return fig

def show(data, x, xaxis, yaxis, title, clipped):
    if clipped:
        imshow(data[:,2:], x=x[2:], xaxis=xaxis, yaxis=yaxis, title=title)
    else:
        imshow(data, x=x, xaxis=xaxis, yaxis=yaxis, title=title)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

Autosaving every 60 seconds


In [2]:
# load mamba
import os
from mamba_lens import HookedMamba
import os
import datetime
model = HookedMamba.from_pretrained("state-spaces/mamba-370m", device='cuda')

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Moving model to device:  cuda


In [55]:
import itertools
import random
import re
# manual patching

# Alice Bob. End is bob
# hypothesize bob stored somewhere in stream, when alice appears again, prompts stored 
# Carol and dennis, see if output is dennis even though

abc_formats = ['ABC DEF']
share_one_abc = ['ABC', 'BAC', 'BCA']
share_one_ade = [x.replace("B", "D").replace("C", "E") for x in share_one_abc]
abc_formats += [a + " " + b for (a,b) in itertools.product(share_one_abc, share_one_ade)]

# share two
share_two_abc = ['ABC', 'ACB', 'CAB']
share_two_ade = [x.replace("C", "D") for x in share_two_abc]
share_two_all = [(a,b) for (a,b) in itertools.product(share_two_abc, share_two_ade)]
abc_formats += [a + " " + b for a,b in share_two_all]
# swap A and B in second one
abc_formats += [a + " " + b.replace("A", "W").replace("B", "A").replace("W","B") for a,b in share_two_all]

# share three
abc_formats += ['ABC ABC']
abc_formats += ['ABC ACB', 'ABC CBA', 'ABC BAC'] # fix one, swap others
abc_formats += ['ABC BCA', 'ABC CAB'] # shift

print(abc_formats)


from test_data import restrict_to_most_common_size, IOI_custom_generator
import test_data


format = """
ABC BC A
ACB AC B""".strip()

IOI_custom_generator(

prompt_uncorrupted, prompt_corrupted, uncorrupted_answer, corrupted_answer = generate_data(format)
print(prompt_uncorrupted)
print(prompt_corrupted)
print(uncorrupted_answer)
print(corrupted_answer)


['ABC DEF', 'ABC ADE', 'ABC DAE', 'ABC DEA', 'BAC ADE', 'BAC DAE', 'BAC DEA', 'BCA ADE', 'BCA DAE', 'BCA DEA', 'ABC ABD', 'ABC ADB', 'ABC DAB', 'ACB ABD', 'ACB ADB', 'ACB DAB', 'CAB ABD', 'CAB ADB', 'CAB DAB', 'ABC BAD', 'ABC BDA', 'ABC DBA', 'ACB BAD', 'ACB BDA', 'ACB DBA', 'CAB BAD', 'CAB BDA', 'CAB DBA', 'ABC ABC', 'ABC ACB', 'ABC CBA', 'ABC BAC', 'ABC BCA', 'ABC CAB']
['Abbey', 'Ada', 'Adelaide', 'Adrian', 'Ag', 'Alberta', 'Alexa', 'Alice', 'Allison', 'Amanda', 'Amber', 'Amy', 'Ana', 'Andrea', 'Andy', 'Angela', 'Anna', 'Anne', 'Annie', 'April', 'Ara', 'Ashley', 'Asia', 'Atlanta', 'Audi', 'Augustine', 'Aurora', 'Austin', 'Bab', 'Barbara', 'Barry', 'Bea', 'Bee', 'Bella', 'Belle', 'Bernie', 'Berry', 'Bert', 'Bess', 'Beth', 'Betty', 'Beverly', 'Billy', 'Bird', 'Blair', 'Blake', 'Bobby', 'Bren', 'Brett', 'Britt', 'Brooks', 'Cal', 'Cam', 'Carey', 'Carolina', 'Caroline', 'Carrie', 'Carroll', 'Cary', 'Casey', 'Cassie', 'Catherine', 'Cecil', 'Cele', 'Chad', 'Charlotte', 'Chelsea', 'Cherry',

In [56]:
# modified from neel nanda's examples

# how should we patch when there are multiple outputs?
# probably just use probability space, sum the probabilities of all the incorrect and correct options

limited_layers = [0, 7, 10, 11, 13, 18, 19, 21, 23, 24, 28, 32, 38, 39]

from tqdm.notebook import tqdm
from functools import partial
from jaxtyping import Float
from transformer_lens.hook_points import HookPoint
from IPython.display import display, clear_output
import ipywidgets
import torch
import plotly.subplots as sp

prompt_uncorrupted_tokens = model.to_tokens(prompt_uncorrupted)
prompt_corrupted_tokens = model.to_tokens(prompt_corrupted)

# logits should be [B,L,V] 
def uncorrupted_logit_minus_corrupted_logit(logits, uncorrupted_answer, corrupted_answer):
    uncorrupted_index = model.to_single_token(uncorrupted_answer)
    corrupted_index = model.to_single_token(corrupted_answer)
    return logits[0, -1, uncorrupted_index] - logits[0, -1, corrupted_index]

# prs should be [B,L,V] 
def uncorrupted_pr_minus_corrupted_pr(prs, uncorrupted_answer, corrupted_answer):
    uncorrupted_index = model.to_single_token(uncorrupted_answer)
    corrupted_index = model.to_single_token(corrupted_answer)
    return prs[0, -1, uncorrupted_index] - prs[0, -1, corrupted_index]

# [B,L,V]
corrupted_logits, corrupted_activations = model.run_with_cache(prompt_corrupted_tokens, only_use_these_layers=limited_layers)
corrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=corrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
corrupted_prs = torch.softmax(corrupted_logits, dim=2)
corrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=corrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)

# [B,L,V]
uncorrupted_logits = model(prompt_uncorrupted_tokens, only_use_these_layers=limited_layers)
uncorrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=uncorrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
uncorrupted_prs = torch.softmax(uncorrupted_logits, dim=2)
uncorrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=uncorrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)

uncorrupted_index = model.to_single_token(uncorrupted_answer)
corrupted_index = model.to_single_token(corrupted_answer)
print(f'uncorrupted prompt\n{prompt_uncorrupted}')
print(f"{repr(uncorrupted_answer)} logit {uncorrupted_logits[0,-1,uncorrupted_index]}")
print(f"{repr(uncorrupted_answer)} pr {uncorrupted_prs[0,-1,uncorrupted_index]}")
print(f"{repr(corrupted_answer)} logit {uncorrupted_logits[0,-1,corrupted_index]}")
print(f"{repr(corrupted_answer)} pr {uncorrupted_prs[0,-1,corrupted_index]}")
print(f'\ncorrupted prompt\n{prompt_corrupted}')
print(f"{repr(uncorrupted_answer)} logit {corrupted_logits[0,-1,uncorrupted_index]}")
print(f"{repr(uncorrupted_answer)} pr {corrupted_prs[0,-1,uncorrupted_index]}")
print(f"{repr(corrupted_answer)} logit {corrupted_logits[0,-1,corrupted_index]}")
print(f"{repr(corrupted_answer)} pr {corrupted_prs[0,-1,corrupted_index]}")

# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
L = len(prompt_uncorrupted_tokens[0])
if len(prompt_corrupted_tokens[0]) != len(prompt_uncorrupted_tokens[0]):
    raise Exception("Prompts are not the same length") # feel free to comment this out, you can patch for different sized prompts its just a lil sus

# diff is logit of uncorrupted_answer - logit of corrupted_answer
# we expect corrupted_diff to have a negative value (as corrupted should put high pr on corrupted_answer)
# we expect uncorrupted to have a positive value (as uncorrupted should put high pr on uncorrupted_answer)
# thus we can treat these as (rough) min and max possible values
min_logit_diff = corrupted_logit_diff
max_logit_diff = uncorrupted_logit_diff

min_pr_diff = corrupted_pr_diff
max_pr_diff = uncorrupted_pr_diff

# make token labels that describe the patch
corrupted_str_tokens = model.to_str_tokens(prompt_corrupted_tokens)
uncorrupted_str_tokens = model.to_str_tokens(prompt_uncorrupted_tokens)
token_labels = []
for index, (corrupted_token, uncorrupted_token) in enumerate(zip(corrupted_str_tokens, uncorrupted_str_tokens)):
    if corrupted_token == uncorrupted_token:
        token_labels.append(f"{corrupted_token}_{index}")
    else:
        token_labels.append(f"{uncorrupted_token}->{corrupted_token}_{index}")

def run_patching(patching_hook_name_func, patching_hook_func, show_options, show_plot=True, normalize=False, **kwargs):
    hook_title = patching_hook_name_func(layer='{layer}', position='{position}')
    print(f"running patching for {hook_title}")
    global patching_result_logits, patching_result_prs # if you want to access it once this is done running
    n_layers = len(limited_layers)
    patching_result_logits = torch.zeros((n_layers, L), device=model.cfg.device)
    patching_result_prs = torch.zeros((n_layers, L), device=model.cfg.device)
    patching_result_logits2 = torch.zeros((n_layers, L), device=model.cfg.device)
    patching_result_prs2 = torch.zeros((n_layers, L), device=model.cfg.device)
    for i, layer in tqdm(list(enumerate(limited_layers))):
        for position in range(L):
            patching_hook_name = patching_hook_name_func(layer=layer, position=position)
            patching_hook = partial(patching_hook_func, layer=layer, position=position)
            # [B,L,V]
            patched_logits = model.run_with_hooks(prompt_uncorrupted_tokens, fwd_hooks=[
                (patching_hook_name, patching_hook)
            ], only_use_these_layers=limited_layers, **kwargs)
            # [B,L,V]
            patched_prs = torch.softmax(patched_logits, dim=2)

            if normalize:
                patched_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=patched_logits,
                                                                             uncorrupted_answer=uncorrupted_answer,
                                                                             corrupted_answer=corrupted_answer)
                if corrupted_answer == uncorrupted_answer:
                    print("warning: both uncorrupted answer and corrupted answer are same, use unnormalized")
                # normalize it so
                # 0 means min_logit_diff (so 0 means that it is acting like the corrupted model)
                # 1 means max_logit_diff (so 1 means that it is acting like the uncorrupted model)
                normalized_patched_logit_diff = (patched_logit_diff-min_logit_diff)/(max_logit_diff - min_logit_diff)
                # now flip them, since most interventions will do nothing and thus act like uncorrupted model, visually its better to have that at 0
                # so now
                # 0 means that it is acting like the uncorrupted model
                # 1 means that it is acting like the corrupted model
                normalized_patched_logit_diff = 1.0 - normalized_patched_logit_diff
                patching_result_logits[i, position] = normalized_patched_logit_diff
                
                # same for pr
                patched_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=patched_prs,
                                                                    uncorrupted_answer=uncorrupted_answer,
                                                                    corrupted_answer=corrupted_answer)
                normalized_patched_pr_diff = 1.0-(patched_pr_diff-min_pr_diff)/(max_pr_diff - min_pr_diff)
                patching_result_prs[i, position] = normalized_patched_pr_diff
            else:
                patching_result_logits[i, position] = patched_logits[0,-1,uncorrupted_index]
                patching_result_logits2[i, position] = patched_logits[0,-1,corrupted_index]
                patching_result_prs[i, position] = patched_prs[0,-1,uncorrupted_index]
                patching_result_prs2[i, position] = patched_prs[0,-1,corrupted_index]

    layer_labels = [str(layer) for layer in limited_layers]
    figs = []
    if normalize:
        if show_options in [SHOW_LOGITS, SHOW_BOTH]:
            figs.append(imshow(patching_result_logits, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Normalized Logit Difference After Patching {hook_title}", font_size=8))
        if show_options in [SHOW_PR, SHOW_BOTH]:
            figs.append(imshow(patching_result_prs, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Normalized Pr Difference After Patching {hook_title}", font_size=8))
    else:
        if show_options in [SHOW_LOGITS, SHOW_BOTH]:
            figs.append(imshow(patching_result_logits, color_continuous_midpoint=None, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Logit of uncorrupted answer {repr(uncorrupted_answer)} after Patching {hook_title}", font_size=8))
            if uncorrupted_answer != corrupted_answer:
                figs.append(imshow(patching_result_logits2, color_continuous_midpoint=None, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Logit of corrupted answer {repr(corrupted_answer)} after Patching {hook_title}", font_size=8))
        if show_options in [SHOW_PR, SHOW_BOTH]:
            figs.append(imshow(patching_result_prs, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Pr of uncorrupted answer {repr(uncorrupted_answer)} after Patching {hook_title}", font_size=8)) 
            if uncorrupted_answer != corrupted_answer:
                figs.append(imshow(patching_result_prs2, show=False, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Pr of corrupted answer {repr(corrupted_answer)} after Patching {hook_title}", font_size=8))

    traces = []
    for fig in figs:
        fig.show()
        fig_trace = []
        for trace in range(len(fig["data"])):
            fig_trace.append(fig["data"][trace])
        traces.append(fig_trace)
    
    #this_figure = sp.make_subplots(rows=2, cols=2)
    #pos = [[1,1], [1,2], [2,1], [2,2]]
    #for i, (fig, trace) in enumerate(zip(figs, traces)):
    #    this_figure.append_trace(traces, row=pos[i][0], col=pos[i][1])
    #this_figure.display()
    '''
    print("logits")
    for pos in [11,15]:
        for layer in range(29, model.cfg.n_layers):
            print(pos, layer, patching_result_logits[layer, pos].item())
    
    print("prs")
    for pos in [11,15]:
        for layer in range(29, model.cfg.n_layers):
            print(pos, layer, patching_result_prs[layer, pos].item())
    '''

def position_patching_hook( # also works for B L E, B L E N, and B L N sized things
    x: Float[torch.Tensor, "B L D"],
    hook: HookPoint,
    position: int,
    layer: int # we don't care about this
) -> Float[torch.Tensor, "B L D"]:
    # only intervene on the specific pos
    corrupted_x = corrupted_activations[hook.name]
    x[:, position, :] = corrupted_x[:, position, :]
    return x

# 'blocks.{layer}.hook_h.{pos}' is the recurrent state of that layer after processing tokens at and before pos position
def h_patching_hook(
    h: Float[torch.Tensor, "B E N"],
    hook: HookPoint,
    position: int,
    layer: int
) -> Float[torch.Tensor, "B E N"]:
    return corrupted_activations[hook.name]


patching_types = {
    'layer input': (lambda layer, position: f'blocks.{layer}.hook_resid_pre', position_patching_hook),
    'normalized input': (lambda layer, position: f'blocks.{layer}.hook_normalized_input', position_patching_hook), 
    'skip': (lambda layer, position: f'blocks.{layer}.hook_skip', position_patching_hook), 
    'in proj': (lambda layer, position: f'blocks.{layer}.hook_in_proj', position_patching_hook), 
    'conv': (lambda layer, position: f'blocks.{layer}.hook_conv', position_patching_hook), 
    'delta 1': (lambda layer, position: f'blocks.{layer}.hook_delta_1', position_patching_hook), 
    'delta 2': (lambda layer, position: f'blocks.{layer}.hook_delta_2', position_patching_hook), 
    'delta': (lambda layer, position: f'blocks.{layer}.hook_delta', position_patching_hook), 
    'A_bar': (lambda layer, position: f'blocks.{layer}.hook_A_bar', position_patching_hook), 
    'B': (lambda layer, position: f'blocks.{layer}.hook_B', position_patching_hook), 
    'B_bar': (lambda layer, position: f'blocks.{layer}.hook_B_bar', position_patching_hook), 
    'C': (lambda layer, position: f'blocks.{layer}.hook_C', position_patching_hook), 
    'ssm input': (lambda layer, position: f'blocks.{layer}.hook_ssm_input', position_patching_hook),
    'h': (lambda layer, position: f'blocks.{layer}.hook_h.{position}', h_patching_hook),
    'y': (lambda layer, position: f'blocks.{layer}.hook_y', position_patching_hook),
    'ssm output': (lambda layer, position: f'blocks.{layer}.hook_ssm_output', position_patching_hook),
    'after skip': (lambda layer, position: f'blocks.{layer}.hook_after_skip', position_patching_hook),
    'out proj': (lambda layer, position: f'blocks.{layer}.hook_out_proj', position_patching_hook),
    'resid post': (lambda layer, position: f'blocks.{layer}.hook_resid_post', position_patching_hook),
}

patching_types_keys = list(patching_types.keys())

def choose_patching_type(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_patching_type.patching_type = change['new'] # hack, gives this function the patching_type attribute

choose_patching_type.patching_type = patching_types_keys[0]

patching_type_dropdown = ipywidgets.Dropdown(
    options=patching_types_keys,
    value=patching_types_keys[0],
    description='Patching Type',
)
patching_type_dropdown.observe(choose_patching_type)
display(patching_type_dropdown)


fast_conv_keys = ['True', 'False']

def choose_fast_conv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_conv.fast_conv = change['new'] == 'True'

choose_fast_conv.fast_conv = fast_conv_keys[0] == 'True'

choose_fast_conv_dropdown = ipywidgets.Dropdown(
    options=fast_conv_keys,
    value=fast_conv_keys[0],
    description='fast conv',
)
choose_fast_conv_dropdown.observe(choose_fast_conv)
display(choose_fast_conv_dropdown)



fast_ssm_keys = ['True', 'False']

def choose_fast_ssm(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_ssm.fast_ssm = change['new'] == 'True'

choose_fast_ssm.fast_ssm = fast_ssm_keys[0] == 'True'

choose_fast_ssm_dropdown = ipywidgets.Dropdown(
    options=fast_ssm_keys,
    value=fast_ssm_keys[0],
    description='fast ssm',
)
choose_fast_ssm_dropdown.observe(choose_fast_ssm)
display(choose_fast_ssm_dropdown)


normalized_keys = ['True', 'False']

def choose_normalized(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_normalized.normalized = change['new'] == 'True'

choose_normalized.normalized = normalized_keys[0] == 'True'

normalized_dropdown = ipywidgets.Dropdown(
    options=normalized_keys,
    value=normalized_keys[0],
    description='normalize',
)
normalized_dropdown.observe(choose_normalized)
display(normalized_dropdown)

SHOW_PR = 'Pr'
SHOW_LOGITS = 'Logits'
SHOW_BOTH = 'Both'
show_options = [SHOW_BOTH, SHOW_LOGITS, SHOW_PR]

def choose_show_options(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_show_options.show_options = change['new']

choose_show_options.show_options = show_options[0]

show_options_dropdown = ipywidgets.Dropdown(
    options=show_options,
    value=show_options[0],
    description='logits or pr',
)
show_options_dropdown.observe(choose_show_options)
display(show_options_dropdown)

def do_patching(arg, show_plot=True):
    with output: # this lets the stuff we output here be visible
        clear_output()
        hook_name_func, hook_func = patching_types[choose_patching_type.patching_type]
        return run_patching(patching_hook_name_func=hook_name_func,
                     patching_hook_func=hook_func,
                     fast_ssm=choose_fast_ssm.fast_ssm,
                     fast_conv=choose_fast_conv.fast_conv,
                     normalize=choose_normalized.normalized,
                     show_options=choose_show_options.show_options,
                     show_plot=show_plot)

patching_button = ipywidgets.Button(description = 'Run Patching')
patching_button.on_click(do_patching)
display(patching_button)

# you can't just display stuff inside a widget callback, you need a wrap any display code in this
output = ipywidgets.Output()
display(output)


uncorrupted prompt
Lately, Violet, Claude, and Leigh had fun at school. Claude and Leigh gave a computer to
' Violet' logit 7.7681708335876465
' Violet' pr 3.4835611932065105e-25
' Claude' logit -28.102453231811523
' Claude' pr 9.196301431824477e-41

corrupted prompt
Lately, Violet, Leigh, and Claude had fun at school. Violet and Leigh gave a computer to
' Violet' logit -24.652851104736328
' Violet' pr 9.491529234365972e-38
' Claude' logit -11.31766128540039
' Claude' pr 5.871316865690532e-32


Dropdown(description='Patching Type', options=('layer input', 'normalized input', 'skip', 'in proj', 'conv', '…

Dropdown(description='fast conv', options=('True', 'False'), value='True')

Dropdown(description='fast ssm', options=('True', 'False'), value='True')

Dropdown(description='normalize', options=('True', 'False'), value='True')

Dropdown(description='logits or pr', options=('Both', 'Logits', 'Pr'), value='Both')

Button(description='Run Patching', style=ButtonStyle())

Output()

Autosaving every 60 seconds
